In [ ]:
pip install tqdm

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#0. Initial Setting

In [ ]:
import numpy as np
from scipy.special import logsumexp
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

theta1 = np.array([1.5,1,-1.9,-1.8,1]) #RS, FC1,FC2, EC

trans_true = [0.9,0.1]
obser_true = [0.2,0.3]
theta23 = trans_true + obser_true

agent = 2
za_dim =(agent+1)*(2**agent)
za_old = np.arange(za_dim,dtype = int)

TimePeriod= 150
SamplePath = 3000
SubSample = 30

# z_old = np.arange(agent+1,dtype=int) #observation
# a1_old = np.arange(2,dtype = int)
# a2_old = np.arange(2,dtype = int)

num_discrete = 50
x_old = np.linspace(0,1,num_discrete)

za_old = np.kron(np.ones(2,dtype=int),np.kron(np.ones(num_discrete,dtype = int),za_old))
x_old = np.kron(np.ones(2,dtype = int),np.kron(x_old,np.ones(za_dim,dtype=int))) #z,a2_old,a1_old,x,a_new
a_new = np.kron(np.arange(2,dtype=int),np.ones(num_discrete*za_dim,dtype=int))

In [ ]:
x_old[100*12:100*24]

array([], dtype=float64)

In [ ]:
#print(za_old//(agent+1))
print((za_old//(agent+1))//2) #a1
print((za_old//(agent+1))%2)  #a2
print(za_old%(agent+1))       #z

# np.hstack([np.hstack([a1_old[i] *2 + a2_old for i in range(2)])[j] *3+z_old for j in range(4)])
print((1*2+1)*3+2) #+ x*za_dim + a_new * za_dim*x_dim

[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
[0 1 2 ... 0 1 2]
11


In [ ]:
haha = np.arange(20)
np.reshape(softmax(np.reshape(haha,[2,-1]),axis=0),-1)

array([4.53978687e-05, 4.53978687e-05, 4.53978687e-05, 4.53978687e-05,
       4.53978687e-05, 4.53978687e-05, 4.53978687e-05, 4.53978687e-05,
       4.53978687e-05, 4.53978687e-05, 9.99954602e-01, 9.99954602e-01,
       9.99954602e-01, 9.99954602e-01, 9.99954602e-01, 9.99954602e-01,
       9.99954602e-01, 9.99954602e-01, 9.99954602e-01, 9.99954602e-01])

#1. Generate Dynamic Part and Policy Part

In [ ]:
def Dynamic(theta23):

  trans_val = theta23[0:2]
  obser_val = theta23[2:4]
  trans = np.zeros([2,2]) #s_t,s_{t+1}
  trans[:,0] = trans_val
  trans[:,1] = 1- trans[:,0]
  
  observe = np.zeros([2,3,3]) #s_{t+1},z_t,z_{t+1}
  for s in range(2):
    observe[s,0,1] = observe[s,1,0] = observe[s,1,2]= observe[s,2,1] = obser_val[s]
    observe[s,0,0] = 1 - observe[s,0,1] -observe[s,0,2]
    observe[s,1,1] = 1 - observe[s,1,0] -observe[s,1,2]
    observe[s,2,2] = 1 - observe[s,2,0] - observe[s,2,1]

  return trans,observe
#print(Dynamic(theta23))

def Reward(theta1,ag_i): # 3*2*2*100*2,2
  rw = np.zeros([(agent+1)*(2**agent)*num_discrete*2,2])#z*a_{t-1}*x_t*a_{t,i},a_{t,-i}
  #for s in range(2):
  if ag_i==1:
    rw[:,0] = ((za_old%(agent+1)) * theta1[0]/((2+0)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2)))*x_old
    rw[:,0] += ((za_old%(agent+1)) * theta1[1]/((2+0)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2)))*(1-x_old)

    rw[:,1] = ((za_old%(agent+1)) * theta1[0]/((2+1)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2)))*x_old
    rw[:,1] += ((za_old%(agent+1)) * theta1[1]/((2+1)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2)))*(1-x_old)

  elif ag_i==2:
    rw[:,0] = ((za_old%(agent+1)) * theta1[0]/((2+0)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2)))*x_old
    rw[:,0] += ((za_old%(agent+1)) * theta1[1]/((2+0)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2)))*(1-x_old)

    rw[:,1] = ((za_old%(agent+1)) * theta1[0]/((2+1)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2)))*x_old
    rw[:,1] += ((za_old%(agent+1)) * theta1[1]/((2+1)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2)))*(1-x_old)

  rw[:,0] = a_new * rw[:,0]
  rw[:,1] = a_new * rw[:,1]

  return rw
#print(Reward(theta1,2)[1200:2400])


def SigmaLambda(za_old,x_old,trans,observe,num_discrete=num_discrete,T = None): #(xt,z_t,a_old,a_new_i),z_{t+1}
  if T==None: #generate function for Q function iteration
      x_new = np.zeros([x_old.shape[0],3])
      sigma = np.zeros([x_old.shape[0],3])
      x_old = np.vstack([x_old,1-x_old])
      for z_new in range(3):
        x_new_tep = trans[:,0].reshape([-1,1])
        x_new_tep = x_new_tep.dot(observe[0,za_old%(agent+1),z_new].reshape([1,-1]))
        x_new_tep = np.sum(x_old * x_new_tep,axis=0)
        #print(x_new_tep.shape)
        sigma[:,z_new] = x_new_tep + np.sum(x_old*(np.reshape(trans[:,1],[-1,1]).dot(observe[1,za_old%(agent+1),z_new].reshape([1,-1]))),
                                            axis=0)
        sigma_0 = np.where(sigma[:,z_new]!=0)
        x_new[:,z_new][sigma_0] = x_new_tep[sigma_0] /sigma[:,z_new][sigma_0] 

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]

  else:      #generate whole belief for recover process
      x_new = np.zeros([T,x_old.shape[0]]) # T, N
      sigma = np.zeros([T-1,x_old.shape[0]]) # T-1 ,N
      x_new[0,:] = x_old

      for t in range(1,T):

        za = za_old[t-1,:]
        z = za_old[t,:]
        
        x_new_tep = (x_new[t-1,:] *trans[0,0] + (1-x_new[t-1,:])*trans[1,0] )* observe[0,za,z]
        sigma[t-1,:] = x_new_tep + (x_new[t-1,:] *trans[0,1] + (1-x_new[t-1,:])*trans[1,1] )* observe[1,za,z]
        x_new[t,:] = x_new_tep/sigma[t-1,:]

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      

  return sigma,x_new,[iterpolate,np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))]
# trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
# print(SigmaLambda(za_old,x_old,trans,observe)[0].shape)

# trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
# sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)

def ValueIteration(theta1,sigma,xiterpolate, num_discrete=num_discrete,beta = 0.95,
                   gamma = 0.5772,thread_inner = 1e-5,thread_outer = 1e-5): #xt*za*a1 or xt*za*a2
  rw1,rw2 = Reward(theta1,1), Reward(theta1,2) #z*a_{t-1}*xt*a_{t,i},a_{t,-i}
  iterpolate,belief_f,belief_c = xiterpolate[0],xiterpolate[1],xiterpolate[2]#z*a_{t-1}*xt*a_{t,i},a_{t,-i}
  x_old = np.linspace(0,1,num=num_discrete)#.reshape([-1,1])
  x_old = np.kron(np.ones(2,dtype = int),np.kron(x_old,np.ones(za_dim,dtype=int))) #z,a2_old,a1_old,x,a_new  
  #x_old = np.vstack([x_old,1-x_old]).T

  pi1_new = 0.5*np.ones(num_discrete*za_dim*2) #xt*za*a1

  pi2_old = pi1_new.copy()
  #count = 0
  #pi1_old = np.zeros([2,2])
  for ol in range(1000):
    pi1_old = pi1_new.copy()
    
    Q1_new = np.zeros(num_discrete*za_dim*2) #xt,za,a1
    Q2_new = np.zeros(num_discrete*za_dim*2) #xt,za,a2

    for il in range(1000):
      Q2_old = Q2_new.copy() #xt,za,a2

      #a1 = 0
      V2_a10 = []
      for z_new in range(3):
        znew_a10= (0 *2+a_new)*3 + z_new
        it0,bf0,bc0 = iterpolate[:,z_new],belief_f[:,z_new],belief_c[:,z_new]
        #print(znew_a10,bf0)
        V2_a10.append( np.stack([it0*Q2_old[bf0*za_dim+znew_a10+0*num_discrete*za_dim] +\
                             (1-it0)*Q2_old[bc0*za_dim+znew_a10+0*num_discrete*za_dim], 
                                 it0*Q2_old[bf0*za_dim+znew_a10+1*num_discrete*za_dim] +\
                             (1-it0)*Q2_old[bc0*za_dim+znew_a10+1*num_discrete*za_dim] ],axis=1))
      V2_a10 =  np.stack(V2_a10,axis=2)

      Q2_tepa10 = rw2[:,0] + beta * np.sum(sigma *(gamma+logsumexp(V2_a10,axis=1)),axis=1)

      #a1 = 1
      V2_a11 = []
      for z_new in range(3):
        znew_a11= (1 *2+a_new)*3 + z_new
        it1,bf1,bc1 = iterpolate[:,z_new],belief_f[:,z_new],belief_c[:,z_new]
        V2_a11.append( np.stack([it1*Q2_old[bf1*za_dim+znew_a11+0*num_discrete*za_dim] +\
                             (1-it1)*Q2_old[bc1*za_dim+znew_a11+0*num_discrete*za_dim], 
                                 it1*Q2_old[bf1*za_dim+znew_a11+1*num_discrete*za_dim] +\
                             (1-it1)*Q2_old[bc1*za_dim+znew_a11+1*num_discrete*za_dim] ],axis=1))
      V2_a11 =  np.stack(V2_a11,axis=2)
      Q2_tepa11 = rw2[:,1] + beta * np.sum(sigma*(gamma+logsumexp(V2_a11,axis=1)),axis=1)

          
      Q2_new = np.kron(np.ones(2),pi1_old[0:num_discrete*za_dim]) * Q2_tepa10 +\
               np.kron(np.ones(2),pi1_old[num_discrete*za_dim:]) * Q2_tepa11
      #print(it0)
      
      #print(il,np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))))
      if np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))) <thread_inner: 
        #print('Q2',il,np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))))
        break

    pi2_new = np.reshape(softmax(np.reshape(Q2_new,[2,-1]),axis=0),-1)   
    count = np.max(np.abs(np.nan_to_num((pi2_old-pi2_new)/pi2_old)))


    pi2_old = pi2_new.copy()
    for il in range(1000):
      Q1_old = Q1_new.copy()

      #a2 = 0
      V1_a20 = []
      for z_new in range(3):
        znew_a20= (a_new *2+0)*3 + z_new
        it0,bf0,bc0 = iterpolate[:,z_new],belief_f[:,z_new],belief_c[:,z_new]
        V1_a20.append( np.stack([it0*Q1_old[bf0*za_dim+znew_a20+0*num_discrete*za_dim] +\
                             (1-it0)*Q1_old[bc0*za_dim+znew_a20+0*num_discrete*za_dim], 
                                 it0*Q1_old[bf0*za_dim+znew_a20+1*num_discrete*za_dim] +\
                             (1-it0)*Q1_old[bc0*za_dim+znew_a20+1*num_discrete*za_dim] ],axis=1))
      V1_a20 =  np.stack(V1_a20,axis=2)

      Q1_tepa20 = rw1[:,0] + beta * np.sum(sigma*(gamma+logsumexp(V1_a20,axis=1)),axis=1)

      #a2 = 1
      V1_a21 = []
      for z_new in range(3):
        znew_a21= (a_new *2+1)*3 + z_new
        it1,bf1,bc1 = iterpolate[:,z_new],belief_f[:,z_new],belief_c[:,z_new]
        V1_a21.append( np.stack([it1*Q1_old[bf1*za_dim+znew_a21+0*num_discrete*za_dim] +\
                             (1-it1)*Q1_old[bc1*za_dim+znew_a21+0*num_discrete*za_dim], 
                                 it1*Q1_old[bf1*za_dim+znew_a21+1*num_discrete*za_dim] +\
                             (1-it1)*Q1_old[bc1*za_dim+znew_a21+1*num_discrete*za_dim] ],axis=1))
      V1_a21 =  np.stack(V1_a21,axis=2)

      Q1_tepa21 = rw1[:,1] + beta * np.sum(sigma*(gamma+logsumexp(V1_a21,axis=1)),axis=1)

        
      Q1_new =np.kron(np.ones(2),pi2_old[0:num_discrete*za_dim]) * Q1_tepa20 +\
               np.kron(np.ones(2),pi2_old[num_discrete*za_dim:]) * Q1_tepa21

      #print(il,np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))))

      if np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))) <thread_inner:
        #print('Q1',il,np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))))
        break

    pi1_new = np.reshape(softmax(np.reshape(Q1_new,[2,-1]),axis=0),-1)  
    
    #print('pi1 pi2',ol,np.max(np.abs(np.nan_to_num((pi1_old-pi1_new)/pi1_old))),count)

    if np.max(np.abs(np.nan_to_num((pi1_old-pi1_new)/pi1_old))) <thread_outer and count<thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return Q1_new,Q2_new,pi1_new,pi2_new

trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)
Q1,Q2,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)

In [ ]:
pi1[0:600]

array([0.14671778, 0.13023735, 0.11581756, 0.1467213 , 0.13159494,
       0.11802793, 0.05949193, 0.05220055, 0.04594575, 0.05949351,
       0.05279924, 0.04690857, 0.14671997, 0.13008746, 0.11554429,
       0.14672348, 0.13145611, 0.1177679 , 0.05949292, 0.05213492,
       0.04582836, 0.05949448, 0.05273841, 0.04679672, 0.14671842,
       0.12992332, 0.11527234, 0.14672194, 0.1313028 , 0.11750907,
       0.05949222, 0.05206308, 0.04571158, 0.05949379, 0.05267126,
       0.04668542, 0.14671688, 0.12976363, 0.11500419, 0.14672041,
       0.13115393, 0.11725398, 0.05949153, 0.0519932 , 0.04559647,
       0.05949311, 0.05260607, 0.04657576, 0.14671696, 0.12960699,
       0.11472138, 0.14672049, 0.13100809, 0.1169836 , 0.05949157,
       0.05192466, 0.04547514, 0.05949314, 0.05254221, 0.04645959,
       0.14671707, 0.12945059, 0.11444942, 0.14672059, 0.13086246,
       0.11672424, 0.05949162, 0.05185624, 0.04535849, 0.05949319,
       0.05247846, 0.04634819, 0.14671723, 0.12929444, 0.11417

#2. Data Generation Process

In [ ]:
trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
#trans-np.diag(np.ones(2))

In [ ]:
##################################Global Parameter#################################
# theta1 = np.array([3,1.9,2]) #RS, FC, EC
# theta23 = 0.2
# agent = 2
# za_dim =(agent+1)*(2**agent)
# za_old = np.arange(z_dim,dtype = int)
# TimePeriod= 150
# SamplePath = 300
# num_discrete = 100
# z_old = np.arange(agent+1,dtype=int) #observation
# a1_old = np.arange(2,dtype = int)
# a2_old = np.arange(2,dtype = int)
###################################################################################
from tqdm import tqdm 

def DataGeneration(theta1 = theta1,theta23=theta23,T=TimePeriod,N=SamplePath,N_sub = SubSample):
  # Assume start from good condition and highest market size(2), agents entry market(1) at time 0

  trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
  sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)#xt,za, a1 or xt,za, a2


  condition = np.zeros([T,N],dtype= int)
  condition[0,:] = np.random.choice(np.array(2,dtype=int),size=N,p=[1/2,1/2])

  market_size = np.zeros([T,N],dtype = int) #observation state
  market_size[0,:] = np.random.choice(np.array(3,dtype=int),size=N,p=[1/3,1/3,1/3])

  belief = np.zeros([T,N])
  belief[0,:] = np.kron(np.linspace(0,1,num=int(N/N_sub)),np.ones(N_sub))
  sigma_val = np.ones([T-1,N])
  belief_f = np.floor(belief[0,:]*(num_discrete-1))/(num_discrete-1)
  belief_c = np.ceil(belief[0,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate = np.zeros(belief_f.shape)
  iterpolate[np.where((belief_f-belief_c)!=0)] = (belief[0,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  action1 = np.ones([T,N],dtype = int)   #agent1's action
  action2 = np.ones([T,N],dtype = int)   #agent2's action
  # za = (np.zeros(N,dtype=int) *2 + np.zeros(N,dtype=int))*3+market_size[0,:]
  # action_mat1 = np.random.random(size=N)
  # p1 = iterpolate * pi1[belief_f,za,0] + (1-iterpolate) * pi1[belief_c,za,0]
  # action1[0,:] = np.int_(p1<=action_mat1)
  
  # action_mat2 = np.random.random(size=N)
  # p2 = iterpolate * pi2[belief_f,za,0] + (1-iterpolate) * pi2[belief_c,za,0]
  # action2[0,:] = np.int_(p2<=action_mat2)


  for t in tqdm(range(1,T)):
    #generate new condition #s_{t+1}
    cd_mat = np.random.random(size=N)
    pc = trans[condition[t-1,:],0]
    condition[t,:] = np.int_(pc<=cd_mat)
   
    # generate new market size #z_{t+1}
    ms_mat = np.random.random(size=N)
    pt = observe[condition[t,:],market_size[t-1,:],:]
    market_size[t,:] = np.int_(pt[:,0]<=ms_mat) +np.int_(pt[:,0]+pt[:,1]<=ms_mat)

    # update new belief #x_t
    za = market_size[t-1,:]
    z = market_size[t,:]
    x_new_tep = (belief[t-1,:] *trans[0,0] + (1-belief[t-1,:])*trans[1,0] )* observe[0,za,z]
    #print(x_new_tep)
    sigma_val[t-1,:] = x_new_tep + (belief[t-1,:] *trans[0,1] + (1-belief[t-1,:])*trans[1,1] )* observe[1,za,z]
    belief[t,:] = x_new_tep/sigma_val[t-1,:]

    belief_f = np.floor(belief[t,:]*(num_discrete-1))/(num_discrete-1)
    belief_c = np.ceil(belief[t,:]*(num_discrete-1))/(num_discrete-1)
    iterpolate = np.zeros(belief_f.shape)
    iterpolate[np.where((belief_f-belief_c)!=0)] = (belief[t,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
    belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

    # generate new action for both agents #a_{t+1}
    za = (action1[t-1,:] *2 + action2[t-1,:])*3+market_size[t,:]
    action_mat1 = np.random.random(size=N)
    p1 = iterpolate * pi1[belief_f*za_dim+za+0] + (1-iterpolate) * pi1[belief_c*za_dim+za+0]
    action1[t,:] = np.int_(p1<=action_mat1)
    
    action_mat2 = np.random.random(size=N)
    p2 = iterpolate * pi2[belief_f*za_dim+za+0]+ (1-iterpolate) * pi2[belief_c*za_dim+za+0]
    action2[t,:] = np.int_(p2<=action_mat2)

  return market_size, action1,action2 ,belief,sigma_val,condition


market_size, action1,action2 ,belief,sigma_val,condition = DataGeneration()
#haha = SigmaLambda()

100%|██████████| 149/149 [00:00<00:00, 2304.98it/s]


In [ ]:
np.sum(condition)/np.prod(condition.shape)

0.4990177777777778

#3.1 Recover Dynamic Part

In [ ]:
theta23

[0.9, 0.1, 0.2, 0.3]

In [ ]:
from scipy.optimize import minimize
ini_x0 = 0.85*np.array(theta23)
print('start from:',ini_x0 )

def func1(theta23,market_size=market_size, action1=action1,action2 = action2,T=TimePeriod ,x0 = belief[0,:]):
  trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
  sigma_val,_,_ = SigmaLambda(market_size,x0,trans,observe,T=T)

  # sigma_val,_,_ = SigmaLambda(None,market_size,x_old,action1,action2,theta23= theta23,num_discrete=num_discrete,T = T)
  #print(observe[market_size[0:-1,:],market_size[1:T,:]])
  res = -np.sum(np.log(sigma_val))
  #print(theta23,res)
  return res

res1 = minimize(func1,ini_x0, bounds=[[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.49999],[0.00001,0.49999]])
#trans_est = np.vstack([res1.x,1-res1.x]).T.reshape([2,2,2,2])
print(res1)

start from: [0.765 0.085 0.17  0.255]
      fun: 322101.75646246824
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.80443749, -2.0430889 ,  4.51691449,  5.34928404])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 145
      nit: 23
   status: 0
  success: True
        x: array([0.86151458, 0.10306746, 0.18857821, 0.29528578])


#3.2 Recover Reward Part

In [ ]:
theta1.reshape(-1)

array([ 1.5,  1. , -1.9, -1.8,  1. ])

In [ ]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
_,belief_val,_ = SigmaLambda(market_size,belief[0,:],trans,observe,T=TimePeriod)
sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val=belief_val,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod,x0 = belief[0,:]):
  #theta1 = theta1.reshape([2,2,2])  
 
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


  belief_f = np.floor(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief_c = np.ceil(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate = np.zeros(belief_f.shape)
  iterpolate[np.where((belief_f-belief_c)!=0)] = (belief_val[1:T,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  p1 = iterpolate*pi1[belief_f*za_dim+za+action1[1:T,:]*num_discrete*za_dim] +\
  (1- iterpolate)*pi1[belief_c*za_dim+za+action1[1:T,:]*num_discrete*za_dim]
  p2 = iterpolate*pi2[belief_f*za_dim+za+action2[1:T,:]*num_discrete*za_dim]+\
  (1- iterpolate)*pi2[belief_c*za_dim+za+action2[1:T,:]*num_discrete*za_dim]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res
# ineq_cons = {'type': 'ineq',
#              'fun' : lambda x: np.array([x[0] - x[1]])}
res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 1.275  0.85  -1.615 -1.53   0.85 ]
[ 1.275  0.85  -1.615 -1.53   0.85 ] 203270.0060851916
[ 1.27500001  0.85       -1.615      -1.53        0.85      ] 203270.00606912404
[ 1.275       0.85000001 -1.615      -1.53        0.85      ] 203270.00606906455
[ 1.275       0.85       -1.61499999 -1.53        0.85      ] 203270.00622345664
[ 1.275       0.85       -1.615      -1.52999999  0.85      ] 203270.00623018268
[ 1.275       0.85       -1.615      -1.53        0.85000001] 203270.00588453194
[ 100.  100. -100. -100.  100.] 10111724.444444455
[ 100.00000001  100.         -100.         -100.          100.        ] 10111724.4444707
[ 100.          100.00000001 -100.         -100.          100.        ] 10111724.444470938
[ 100.          100.          -99.99999999 -100.          100.        ] 10111724.444196131
[ 100.          100.         -100.          -99.99999999  100.        ] 10111724.444164932
[ 100.          100.         -100.         -100.          100.00000001] 101117

In [ ]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

trans,observe = Dynamic(res1.x)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
_,belief_val,_ = SigmaLambda(market_size,belief[0,:],trans,observe,T=TimePeriod)
sigma,x_new,xiterpolate = SigmaLambda(za_old,x_old,trans,observe)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val=belief_val,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod,x0 = belief[0,:]):
  #theta1 = theta1.reshape([2,2,2])  
 
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


  belief_f = np.floor(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief_c = np.ceil(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate = np.zeros(belief_f.shape)
  iterpolate[np.where((belief_f-belief_c)!=0)] = (belief_val[1:T,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  p1 = iterpolate*pi1[belief_f*za_dim+za+action1[1:T,:]*num_discrete*za_dim] +\
  (1- iterpolate)*pi1[belief_c*za_dim+za+action1[1:T,:]*num_discrete*za_dim]
  p2 = iterpolate*pi2[belief_f*za_dim+za+action2[1:T,:]*num_discrete*za_dim]+\
  (1- iterpolate)*pi2[belief_c*za_dim+za+action2[1:T,:]*num_discrete*za_dim]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res
# ineq_cons = {'type': 'ineq',
#              'fun' : lambda x: np.array([x[0] - x[1]])}
res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 1.275  0.85  -1.615 -1.53   0.85 ]
[ 1.275  0.85  -1.615 -1.53   0.85 ] 203369.76723540708
[ 1.27500001  0.85       -1.615      -1.53        0.85      ] 203369.7672214051
[ 1.275       0.85000001 -1.615      -1.53        0.85      ] 203369.76721661567
[ 1.275       0.85       -1.61499999 -1.53        0.85      ] 203369.76737509965
[ 1.275       0.85       -1.615      -1.52999999  0.85      ] 203369.76738195997
[ 1.275       0.85       -1.615      -1.53        0.85000001] 203369.76703278848
[ 100.  100. -100. -100.  100.] 10111724.444444442
[ 100.00000001  100.         -100.         -100.          100.        ] 10111724.444466904
[ 100.          100.00000001 -100.         -100.          100.        ] 10111724.444474733
[ 100.          100.          -99.99999999 -100.          100.        ] 10111724.444196125
[ 100.          100.         -100.          -99.99999999  100.        ] 10111724.444164928
[ 100.          100.         -100.         -100.          100.00000001] 1011